# Anacondaでopencvを使う方法 / How to use OpenCV in Anaconda



1. Anaconda Navigatorで左メニューのEnvironmentsを押し、Create+を押して、opencvという名前の新しい環境を作る。環境とは、Pythonライブラリ(パッケージ)を集めたもの。標準はroot環境だが、opencvは最新環境(3.6)では動かないので、Python3.5を選択する。
2. 最初は最小限(10個)のパッケージしか含まれていない。
2. opencvの緑の三角">"を押してOpen Terminalするとターミナルが開く。
3. その中で、OpenCVに必要なものをインストールする。
       conda install -c https://conda.binstar.org/menpo opencv3 spyder
   これにより、新しい環境(Python3.5, OpenCV, Spyder入り)の準備ができた。今後、また別の開発環境が欲しくなった時も同じ手順を踏む。
5. Anaconda Navigatorに戻り、左メニュー「Home」でアプリ起動画面を表示する。「Applications on」でrootの代わりにopencvを選ぶと、使えるアプリケーションが更新される。これらのアプリケーションすべてで、OpenCVが利用できる。


1. Press "Environments" in the left menu of the Anaconda Navigator, press "Create[+]" button at the bottom, and make a new environment of Python version 3.5 with a new name "opencv".  An environment is a collection of packages (libraries).  The standard environment is named "root" and is working on Python version 3.6, but OpenCV library work on Python version 3.5 only, so we prepare another environment.
2. Initially, only 10 packages are included in the opencv environment.
3. Press the green triangle at "opencv" and select "Open Terminal".
4. In the terminal, type the following command to install the packages required to OpenCV.
        conda install -c https://conda.binstar.org/menpo opencv3 spyder
5. Back to Anaconda Navigator, press "Home" in the left menu to go back the home screen, and select "opencv" in the "Applications on" selector.  It refreshes the menu item of the Navigator.  You can now use OpenCV in Jupyter and spyder.

### Notes
* https://anaconda.org/menpo/opencv3 に一応書かれているがそのままでは動かない。
* https://rivercitylabs.org/up-and-running-with-opencv3-and-python-3-anaconda-edition/ が役に立った。

# OpenCVでとにかく何か表示してみよう。

> OpenCV（オープンシーヴィ、英語: Open Source Computer Vision Library）とはインテルが開発・公開したオープンソースのコンピュータビジョン向けライブラリ[1]。2009年にWillow Garage（ウィロー・ガレージ）に開発が移管された後、2015年現在はItseezがメンテナンスを行なっている[2]。なお、2016年5月26日にインテルがItseezを買収することが発表された。[Wikipedia](https://en.wikipedia.org/wiki/OpenCV)

> OpenCV (Open Source Computer Vision) is a library of programming functions mainly aimed at real-time computer vision.[2] Originally developed by Intel, it was later supported by Willow Garage and is now maintained by Itseez.[1] The library is cross-platform and free for use under the open-source BSD license. [Wikipedia](https://en.wikipedia.org/wiki/OpenCV)

たぶん今のPCならカメラを内蔵しているだろうから、カメラから直接とりこむのが一番てっとりばやい。

Almost all note PC has a camera for Skyping.  We utilize it to capture the sample image.

In [ ]:
import cv2
camera = cv2.VideoCapture(0)  #Device ID of the camera would be 0

ret, image = camera.read()   #Capture a frame from the camera.
cv2.imshow("Hello!", image)  #Show the image in a window.
key = cv2.waitKey(0)         #Wait until a key is pressed in the window.
image

imageの中身に注目して下さい。整数値の三つ組がたくさんはいった,3次元のnumpy arrayになっています。
3つ組の数字はRGB(三原色)を表現していて、写真の画素一つ一つの色がすべて含まれています。
それが行ごとにまとめられ、すべての行をまとめたものが1枚の写真になります。
つまり、numpyのarrayの操作さえ知っていれば、写真を自由に操作できる、ということになります。

See the content of the `image` variable. It is a three-dimensional numpy array of integers.
The innermost triplets express the intensities of the three primitive colors, blue, green, and red, at a single pixel.
Pixels are bundled to make a row, and the rows are budled to make the whole image.
That is, if you know how to access and modify the numpy array, you can also process the image.

`row`行目のピクセルの色は、`image[row]`あるいは`image[row,:,:]`で参照できます。

Colors of the pixels at the `row` are referred to as `image[row]` or `image[row,:,:]`.

In [ ]:
image[100,:,:]

`row`行目、`col`列目のピクセルの色は、`image[row,col]`あるいは`image[row,col,:]`で参照できます。

Color of the pixel at (`row`, `col`) is referred to as `image[row,col]` or `image[row,col,:]`.

In [ ]:
image[100,200,:]

`row`行目、`col`列目のピクセルの赤色の強度は、`image[row,col,2]`で参照できます。色の順番はRGBではなくBGRです。

The red strength of the pixel at (`row`, `col`) is referred to as `image[row,col,2]`. Note that color order is not RGB but BGR (blue is 0, green is 1, red is 2).

In [ ]:
image[100,200,2]

画像は`imwrite()`関数を使って保存できます。

The obtained image can be saved with `imwrite()` function.

In [ ]:
cv2.imwrite("test.jpg", image)

上の例では1枚の写真が得られるだけですが、これをくりかえせば、ムービーになります。
画像解像度が高すぎる場合は、resize関数で小さくして下さい。

In the example above, only one image is obtained. If you repeat the process with a loop, you get a movie.
If the picture size is too large to be displayed, use `resize()` function.

In [ ]:
import cv2
camera = cv2.VideoCapture(0)  #Device ID of the camera would be 0

while cv2.waitKey(1) < 0:
    ret, image = camera.read()
    image = cv2.resize(image, (640,360))
    cv2.imshow("Hello!", image)

#Close all the windows.  
cv2.destroyAllWindows()
cv2.waitKey(1)

## 簡単な画像処理
色の明るさは0〜255の整数で表現されていますから、色調を完全に反転させるには、255から引きます。

カメラからとりこんだ画像を、ウィンドウに表示する前に、処理をすれば、処理後の画像を見ることができます。
もちろん、2つウィンドウをひらいて、両方見比べることも簡単です。

Color intensities are expressed by the integer number between 0 (dark) and 255 (bright). Therefore, if you want to invert the colors, subtract them from 255.

In the following sample, the original and inverted images are shown in a different window.

In [ ]:
import cv2
camera = cv2.VideoCapture(0)  #Device ID of the camera would be 0

while cv2.waitKey(1) < 0:
    ret, image = camera.read()
    image = cv2.resize(image, (640,360))
    reversed_image = 255 - image #reverse the colors (numpy operation)

    cv2.imshow("Hello!", image)
    cv2.imshow("!olleH", reversed_image)  #Show the reversed image

#Close all the windows.  
cv2.destroyAllWindows()
cv2.waitKey(1)

赤だけを反転するには? ここでもnumpyのarray操作が使えます。

Can we then invert the red component only?  Numpy is useful for such an operation. 

In [ ]:
import cv2
camera = cv2.VideoCapture(0)  #Device ID of the camera would be 0

while cv2.waitKey(1) < 0:
    ret, image = camera.read()
    image = cv2.resize(image, (640,360))
    image2 = image.copy() #firstly make the copy of the image
    image2[:,:,2] = 255 - image2[:,:,2]  #Color order in OpenCV is not RGB but BGR.

    cv2.imshow("Hello!", image)
    cv2.imshow("!olleH", image2)  #Show the reversed image

#Close all the windows.  
cv2.destroyAllWindows()
cv2.waitKey(1)

上下を入れかえたいなら、行を反転すればいいのです。

If you want to make the image upside-down, just invert the order of image rows.

In [ ]:
import cv2
camera = cv2.VideoCapture(0)  #Device ID of the camera would be 0

while cv2.waitKey(1) < 0:
    ret, image = camera.read()
    image = cv2.resize(image, (640,360))
    upsidedown = image[::-1,:,:] 

    cv2.imshow("Hello!", image)
    cv2.imshow("!olleH", upsidedown)  #Show the reversed image

#Close all the windows.  
cv2.destroyAllWindows()
cv2.waitKey(1)

### 練習問題1 Exercise 1
では、左右を反転するにはどうしたらいいでしょうか??

Invert the images horizontally instead of vertically.  It makes your PC a mirror.

## 少し高度な画像処理 More tricks
瞬間瞬間の画像を加工するのではなく、時間変化をとらえてみましょう。以下の例では、直前の画像をlastimage変数に保存しておき、absdiff関数で画像の差分を作ります。

Instead of processing an instantaneous image, let us process the difference.  In the following example, the captured image is stored in `lastimage` and compare it with the new capture.

In [2]:
import cv2
camera = cv2.VideoCapture(0)  #Device ID of the camera would be 0

ret, image = camera.read()   #Obtain the first image
lastimage = cv2.resize(image, (640,360))  #and resize it.
while cv2.waitKey(1) < 0:
    ret, image = camera.read()
    image = cv2.resize(image, (640,360))
    diff = cv2.absdiff(image, lastimage)
    cv2.imshow("Hello!", diff)
    lastimage = image

#Close all the windows.  
cv2.destroyAllWindows()
cv2.waitKey(1)

-1

爬虫類や両生類は、動いているものしか見えないと言います。たぶん彼らの見ている世界はこんな感じなのでしょう。

It is said that reptiles and amphibians only see the moving things. Perhaps they are looking the world like this.

## 高度な画像処理 Advanced image processing
OpenCVは最先端の画像処理ライブラリなので、単なる画像の編集以上のことができます。私たちは画像処理の専門家ではないので、技術的な内容まではよくわかりませんが、とにかく利用することならできます。

ここでは、最近のカメラのように、映像の中に顔を見付けだす実験をやってみます。といっても、自分ではその方法が見当もつかないので、Googleに頼ります。

Googleで"OpenCV python face detection" "OpenCV python 顔認識" などのキーワードで検索すると、10行程度のコードを書くだけで顔認識してくれることがわかります。今回は、[http://qiita.com/wwacky/items/98d8be2844fa1b778323](http://qiita.com/wwacky/items/98d8be2844fa1b778323) のコードを読み、上で書いたプログラムに組み込んで、動画からリアルタイムで顔を認識するようにしてみましょう。

Since OpenCV is a state-of-the-art image processing library, it can do more than just editing images. We are not experts in image processing, so we do not know the technical content well, but we can do it anyway.

Let us try to detect the faces in the captured image like a recent digital cameras and smart phones.

Giving the keywords such as "OpenCV python face recognition" on Google, you can get the codes for face recognition.  They are only 10 lines of Python.  For this time, I read the code of [http://qiita.com/wwacky/items/98d8be2844fa1b778323] (http://qiita.com/wwacky/items/98d8be2844fa1b778323) and incorporate it into the program written above.

コードをそのままコピーするのははばかられるので、要点だけ書くと、

Essential parts of the code are following.

    #HAAR分類器の顔検出用の特徴量。顔らしさの指標。これはOpenCVのサンプルコードに付属しているものをそのまま利用。
    #Feature collection of the faces for HAAR classifier. It is contained in the sample code for OpenCV.
    cascade_path = "haarcascade_frontalface_alt.xml"

    #グレースケール変換。モノクロ画像で認識するらしい。
    #この部分はOpenCV3で書き方が変わったらしい。
    #It converts a color image into a gray image.
    image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    #カスケード分類器の特徴量を取得する。ループの外で一回だけ実行するだけでよさそう。
    #Initializer for the feature detector.  Must be executed only once (outside the loop)
    cascade = cv2.CascadeClassifier(cascade_path)

    #物体認識（顔認識）の実行。facerectには、顔を含む領域の長方形(複数)が入る。
    #Feature detection. It returns the list of rectangles in which faces are detected.
    facerect = cascade.detectMultiScale(image_gray, scaleFactor=1.1, minNeighbors=1, minSize=(1, 1))

    #imageに長方形を描きこむ関数。
    #It draws a rectangle on the given image.
    cv2.rectangle(image, tuple(rect[0:2]),tuple(rect[0:2]+rect[2:4]), color, thickness=2)

これらを、上のコードに埋めこみます。

Let us embed them in our program.

In [ ]:
import cv2
camera = cv2.VideoCapture(0)  #Device ID of the camera would be 0

#HAAR分類器の顔検出用の特徴量。顔らしさの指標。これはOpenCVのサンプルコードに付属しているものをそのまま利用。
cascade_path = "haarcascade_frontalface_alt2.xml"
#カスケード分類器の特徴量を取得する。ループの外で一回だけ実行するだけでよさそう。
cascade = cv2.CascadeClassifier(cascade_path)

while cv2.waitKey(1) < 0:
    ret, image = camera.read()
    image = cv2.resize(image, (640,360))
    #グレースケール変換。モノクロ画像で認識するらしい。
    image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    #物体認識（顔認識）の実行。facerectには、顔を含む領域の長方形(複数)が入る。
    facerect = cascade.detectMultiScale(image_gray, scaleFactor=1.3, minNeighbors=2)

    for (x,y,w,h) in facerect:
        #imageに長方形を描きこむ関数。
        cv2.rectangle(image, (x,y), (x+w,y+h), (0,255,0), thickness=2)

    cv2.imshow("Hello!", image)

#Close all the windows.  
cv2.destroyAllWindows()
cv2.waitKey(1)

こういうプログラムは何に使えるの? 何でしょうね。例えば、

* 顔認識は、顕微鏡写真から、細胞の核の個数を数えたりできるでしょう。その場合、特徴量ファイルを細胞核用に準備する必要はあります。
* 動きの検出は、遅い反応や、まれにしかおこらない現象を監視するのに使えます。もちろん、防犯カメラにも。

OpenCVのすべての機能はとても紹介しきれませんが、画像処理を専門としている研究者たちが、OpenCVを使って新しい画像処理方法を見付けたり、逆に新しいアルゴリズムをOpenCVに追加したりしています。OpenCVを使えれば、それらの最新の技術を自分の研究にとりこんでいけるのです。


How can we use them for our research?

* Face recognition can count the number of cell nuclei from the photomicrograph. In that case, you need to prepare the feature file for the cell nucleus.
* Motion detection can be used to monitor slow reactions or rare phenomena. Of course, also for security cameras.

Researchers who specialize in image processing use OpenCV to find a new image processing method and add new algorithms to OpenCV. With OpenCV, you can incorporate those latest technologies into your own research.